# Astrophysics Chat Application 

In [1]:
from ssec_tutorials import OLMO_MODEL

In [2]:
from pathlib import Path
from qdrant_client import QdrantClient
from uuid import uuid4

In [5]:
import panel as pn
from langchain.llms import LlamaCpp
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.callbacks import CallbackManager
from langchain_core.prompts import PromptTemplate
from langchain_community.vectorstores import Qdrant
from langchain.embeddings import HuggingFaceEmbeddings

In [14]:
repo_root = Path("../../resources/data/").resolve()

# template = """<|user|>
# You are an astrophysics expert. Answer the question based only on the following context:

# {context}

# Question: {question}
# <|user|>"""
qdrant_path = repo_root / "scipy_qdrant"
qdrant_collection = "arxiv_astro-ph_abstracts"

# prompt = PromptTemplate.from_template(template)
embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

pn.extension()

model_path = OLMO_MODEL

In [20]:
qdrant_path

PosixPath('/Users/a42/Code/uw-ssec/tutorials/resources/data/scipy_qdrant')

In [17]:
@pn.cache
def get_vector_store():
    # If the Qdrant Vector Database Collection already exists, load it
    client = QdrantClient(path=str(qdrant_path))
    db = Qdrant(
        client=client,
        collection_name=qdrant_collection,
        embeddings=embedding
    )
    return db

In [18]:
db = get_vector_store()

In [19]:
db

In [24]:
def get_chain(callbacks):
    retriever = db.as_retriever(callbacks=callbacks, search_type="mmr", search_kwargs={"k": 3})
    # Callbacks support token-wise streaming
    callback_manager = CallbackManager(callbacks)
    model = LlamaCpp(
        model_path=str(model_path),
        callback_manager=callback_manager,
        temperature=0.8,
        #n_ctx=2048,
        max_tokens=512,
        verbose=False,
        echo=False
    )
    prompt = PromptTemplate.from_template(
        template=model.client.metadata['tokenizer.chat_template'],
        template_format="jinja2"
    )

    def create_format(input_dict):
        context = input_dict.get('context')
        question = input_dict.get('question')
        return dict(
            add_generation_prompt=True,
            messages=[
                {"role": "user", "content": f"""
                    You are an astrophysics expert. Answer the question based only on the following context:

                    {context}

                    Question: {question}"""
                }
            ])

    def format_docs(docs):
        text = "\n\n".join([d.page_content for d in docs])
        return text

    def hack(docs):
        # https://github.com/langchain-ai/langchain/issues/7290
        for callback in callbacks:
            callback.on_retriever_end(docs, run_id=uuid4())
        return docs

    return (
        # NOTE: THIS BREAKS HERE... Can't quite pass in the piping in dict!
        {"context": retriever | hack | format_docs, "question": RunnablePassthrough()}
        | create_format
        | prompt
        | model
    )

In [25]:
async def callback(contents, user, instance):
    callback_handler = pn.chat.langchain.PanelCallbackHandler(instance, user='OLMo', avatar='🌳')
    # Not return the result at the end of the generation
    # this prevents the model from repeating the result
    callback_handler.on_llm_end = lambda response, *args, **kwargs: None
    chain = get_chain(callbacks=[callback_handler])
    response = await chain.ainvoke(contents)

In [26]:
pn.chat.ChatInterface(callback=callback).servable()

ChatInterface(_button_data={'send': _ChatButtonData(i...}, _buttons={'send': Button(align='cen...}, _input_container=Row, _input_layout=Row, _placeholder=ChatMessage, _widgets={'ChatAreaInput': ChatArea...}, callback=<function callback a..., show_button_name=True, sizing_mode='stretch_width', widgets=[ChatAreaInput(css_classes...])

In [29]:
from uuid import uuid4

import requests

from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma

import panel as pn

TEXT = "https://raw.githubusercontent.com/langchain-ai/langchain/master/docs/docs/modules/state_of_the_union.txt"

TEMPLATE = """Answer the question based only on the following context:

{context}

Question: {question}
"""

pn.extension(design="material")

prompt = ChatPromptTemplate.from_template(TEMPLATE)


@pn.cache
def get_vector_store():
    full_text = requests.get(TEXT).text
    text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
    texts = text_splitter.split_text(full_text)
    embeddings = OpenAIEmbeddings()
    db = Chroma.from_texts(texts, embeddings)
    return db


db = get_vector_store()


def get_chain(callbacks):
    retriever = db.as_retriever(callbacks=callbacks)
    model = ChatOpenAI(callbacks=callbacks)

    def format_docs(docs):
        text = "\n\n".join([d.page_content for d in docs])
        return text

    def hack(docs):
        # https://github.com/langchain-ai/langchain/issues/7290
        for callback in callbacks:
            callback.on_retriever_end(docs, run_id=uuid4())
        return docs

    return (
        {"context": retriever | hack | format_docs, "question": RunnablePassthrough()}
        | prompt
        | model
    )


async def callback(contents, user, instance):
    callback_handler = pn.chat.langchain.PanelCallbackHandler(instance)
    chain = get_chain(callbacks=[callback_handler])
    response = await chain.ainvoke(contents)
    return response.content


pn.chat.ChatInterface(callback=callback).servable()

ChatInterface(_button_data={'send': _ChatButtonData(i...}, _buttons={'send': Button(align='cen...}, _input_container=Row, _input_layout=Row, _placeholder=ChatMessage, _widgets={'ChatAreaInput': ChatArea...}, callback=<function callback a..., design=<class 'panel.theme.materi..., show_button_name=True, sizing_mode='stretch_width', widgets=[ChatAreaInput(css_classes...])